In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 1000
# mpl.rcParams['legend.fontsize'] = 15
# params = {'legend.fontsize': 20,
#           'legend.handlelength': 2}
# plot.rcParams.update(params)

In [4]:
from algorithms.moo.nsga2 import NSGA2
from constraints.as_obj import ConstraintsAsObjective
from operators.survival.rank_and_crowding.classes import (
    RankAndCrowding,
    MyConstrRankAndCrowding,
    ParallelConstrRankAndCrowding,
    MyConstrRankAndCrowding2,
)
from optimize import minimize
from visualization.scatter import Scatter
from pymoo.problems import get_problem

In [5]:
from algorithms.moo.nsga2 import binary_tournament, constr_binary_tournament
from operators.selection.tournament import TournamentSelection


core_moea1 = NSGA2(pop_size=500, survival=RankAndCrowding(crowding_func="2nn"),
                   eliminate_duplicates=True)
core_moea2 = NSGA2(
    pop_size=500, selection=TournamentSelection(func_comp=constr_binary_tournament), survival=MyConstrRankAndCrowding(crowding_func="2nn"), eliminate_duplicates=True)
core_moea3 = NSGA2(
    pop_size=500, survival=ParallelConstrRankAndCrowding(crowding_func="2nn"), eliminate_duplicates=True)
core_moea4 = NSGA2(
    pop_size=500, survival=MyConstrRankAndCrowding2(crowding_func="2nn"), eliminate_duplicates=True)

In [6]:
import time


def performance_timer(func):
    """
    A decorator to measure and print the execution time of a function.
    """

    def wrapper(*args, **kwargs):
        start_time = time.time()  # Capture the start time
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # Capture the end time
        duration = end_time - start_time  # Calculate duration
        print(f"{func.__name__} executed in {duration:.6f} seconds")
        return result

    return wrapper


@performance_timer
def minimize_(*args, **kwargs):
    return minimize(*args, **kwargs)

In [7]:
from pymoo.indicators.gd import GD
from pymoo.indicators.gd_plus import GDPlus
from pymoo.indicators.igd import IGD
from pymoo.indicators.igd_plus import IGDPlus
from pymoo.indicators.hv import HV

metric_classes = {
    "Generational Distance (GD)": GD,
    "Generational Distance Plus (GD+)": GDPlus,
    "Inverted Generational Distance (IGD)": IGD,
    "Inverted Generational Distance Plus (IGD+)": IGDPlus,
    # "Hypervolume": HV,
}


def performance_indicators(pf, results):
    # Pre-configuration for each metric, handle special cases like HV here.
    pf_calculators = [
        (
            metric_class(ref_point=np.array([0, 0]))
            if metric_name == "Hypervolume"
            else metric_class(pf)
        )
        for metric_name, metric_class in metric_classes.items()
    ]

    def cal_acc_metric(sols):
        results_matrix = np.empty((len(sols), len(metric_classes)), dtype=object)

        for idx, sol in enumerate(sols):
            # Ensure `sol` is a 2D array with the correct shape [n_samples, n_features]
            sol = np.atleast_2d(sol)
            if sol.shape[0] == 1 and sol.shape[1] != len(pf):
                # This implies sol is a single sample with multiple features, which is the correct format
                pass
            elif sol.shape[1] == 1 and sol.shape[0] != len(pf):
                # This implies sol has the wrong orientation; it's many samples of a single feature
                sol = sol.T

            for metric_idx, calculator in enumerate(pf_calculators):
                try:
                    results_matrix[idx, metric_idx] = calculator(sol)
                except Exception as e:
                    # print(
                    #     f"Error calculating {list(metric_classes.keys())[metric_idx]}: {e}")
                    results_matrix[idx, metric_idx] = np.nan

        return [
            (np.mean(results_matrix[:, i]), np.median(results_matrix[:, i]))
            for i in range(len(pf_calculators))
        ]

    return [cal_acc_metric(rs) for rs in results]

In [8]:
from constraints.as_obj import (
    CVRAsObjective,
    CDFAsObjective,
    CVRAsObjective2,
    CDFAsObjective2,
)


@performance_timer
def compare_on(problem, n_gen=300):
    # Configuration for each optimization strategy
    configurations = [
        (problem, core_moea1, ("n_gen", n_gen), {"seed": 1, "verbose": False}),
        # (ConstraintsAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CDFAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        (ConstraintsAsObjective(problem),
         core_moea2, ("n_gen", n_gen), {"seed": 1}),
        (
            CVRAsObjective(problem),
            core_moea2,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
        (
            CDFAsObjective(problem),
            core_moea2,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
        (
            CVRAsObjective2(problem),
            core_moea4,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
        (
            CDFAsObjective2(problem),
            core_moea4,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
    ]

    results = []
    for config in configurations:
        result = minimize_(*config[:-1], **config[-1])
        results.append(result)
    return results


def test(problem_name, n_gen=300, *args):
    if "dascmop" in problem_name:
        problem = get_problem(problem_name, args[0])
    elif "modact" in problem_name:
        problem = get_problem(problem_name, args[0])
    else:
        problem = get_problem(problem_name)
    pf = problem.pareto_front()
    print(
        f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
    )

    results = compare_on(problem, n_gen=n_gen)

    # Check if any of the results is None
    if any(result is None for result in results):
        print("One or more optimization runs failed to return a result.")
        # Return None in place of results to indicate failure
        return pf, [None] * len(results)

    # Prepare data for performance indicators, safely handling cases where result.F might be None or incorrectly shaped
    prepared_data = []
    for i, result in enumerate(results):
        if result and hasattr(result, "F") and result.F is not None:
            standard_shape_1 = pf.shape[1]
            if standard_shape_1 > 1:
                prepared_data.append(result.F[:, :standard_shape_1])
            else:
                prepared_data.append(result.F[:standard_shape_1])
            # i == 0 this NGSA without additional objective
            # if i == 0:
            #     prepared_data.append(result.F)
            # if i == 1 or i == 2 or i == 3:
            #     # if True:
            #     prepared_data.append(result.F[:, :-1])
            # else:
            #     prepared_data.append(result.F[:, :-2])

        else:
            # Use None or an appropriate placeholder if result is invalid
            print(f"algo {i} not converge")
            prepared_data.append(None)

    # Ensure all data is valid before calculating performance indicators
    if all(data is not None for data in prepared_data):
        performance_results = performance_indicators(pf, prepared_data)
        for i, metric in enumerate(metric_classes.keys()):
            print(
                f"{metric}: " +
                " | ".join(f"{perf[i]}" for perf in performance_results)
            )
    else:
        print("Performance indicators could not be calculated due to invalid data.")

    return pf, *results

# BNH


In [12]:
pf, *results = test("bnh", n_gen=500)
# pf, results[0].F, results[1].F, results[2].F

Problem <bound method Problem.name of <pymoo.problems.multi.bnh.BNH object at 0x123bd17c0>>: n_var=2 n_obj=2 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 33.979668 seconds
minimize_ executed in 27.298818 seconds
minimize_ executed in 23.975899 seconds
minimize_ executed in 24.605809 seconds
minimize_ executed in 25.391918 seconds
minimize_ executed in 26.163589 seconds
compare_on executed in 161.417430 seconds
Generational Distance (GD): (0.3955801905800464, 0.36537997709364434) | (0.38865773494226397, 0.3595988039138314) | (0.37692217775286263, 0.3467947020225375) | (0.37692217775286263, 0.3467947020225375) | (0.38865773494226397, 0.3595988039138314) | (0.38865773494226397, 0.3595988039138314)
Generational Distance Plus (GD+): (0.15352033309931573, 0.12205068826695564) | (0.15528224754605646, 0.11774597594042291) | (0.1553695406199809, 0.12204710233519211) | (0.1553695406199809, 0.12204710233519211) | (0.15528224754605646, 0.11774597594042291) | (0.15528224754605646, 0.117745975

In [13]:
pf, *results = test("srn", n_gen=500)
# pf, results[0].F, results[1].F, results[2].F

Problem <bound method Problem.name of <pymoo.problems.multi.srn.SRN object at 0x12431e1f0>>: n_var=2 n_obj=2 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 29.891158 seconds
minimize_ executed in 25.993197 seconds
minimize_ executed in 25.174025 seconds
minimize_ executed in 25.359488 seconds
minimize_ executed in 25.960216 seconds
minimize_ executed in 27.200060 seconds
compare_on executed in 159.579711 seconds
Generational Distance (GD): (2.3100474871032426, 0.8493599682657973) | (2.4628071232500504, 0.8762876821522878) | (2.590247647450877, 0.8388324186401093) | (2.433273321925067, 0.8711026572653933) | (2.3925961137044847, 0.8571244346737981) | (2.434953539458155, 0.839686454754902)
Generational Distance Plus (GD+): (1.8233481459752843, 0.61912086978068) | (1.9485459800052372, 0.6573777236593799) | (2.0534621311754986, 0.6094271634741517) | (1.9256857718509994, 0.642348273252999) | (1.8903920459119932, 0.6306488478657464) | (1.9217479300831906, 0.6136783614355323)
Inverted Gene

In [14]:
pf, *results = test("osy", n_gen=500)
# pf, results[0].F, results[1].F, results[2].F

Problem <bound method Problem.name of <pymoo.problems.multi.osy.OSY object at 0x12cc80a30>>: n_var=6 n_obj=2 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 27.746738 seconds
minimize_ executed in 24.809448 seconds
minimize_ executed in 23.297174 seconds
minimize_ executed in 22.006264 seconds
minimize_ executed in 24.939754 seconds
minimize_ executed in 25.920528 seconds
compare_on executed in 148.721335 seconds
Generational Distance (GD): (1.9147198212554641, 0.6001596387878056) | (1.315079864288538, 0.4286056061033915) | (1.4883277953959553, 0.4514969372956971) | (1.4761179491943652, 0.454181911744026) | (1.3965259056109796, 0.4231087264346832) | (1.7002208189778034, 0.5876128267450704)
Generational Distance Plus (GD+): (0.9617010659743999, 0.18978408747929087) | (0.42945562038595864, 0.16101320288608) | (0.6511934037953087, 0.18758210621564686) | (0.6524146944915625, 0.17898120156991837) | (0.5613472621182618, 0.17335147247755955) | (0.6845740824361682, 0.18980523858948573)
Inve

In [16]:
pf, *results = test("tnk", n_gen=500)
# pf, results[0].F, results[1].F, results[2].F

Problem <bound method Problem.name of <pymoo.problems.multi.tnk.TNK object at 0x124373850>>: n_var=2 n_obj=2 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 29.420644 seconds
minimize_ executed in 25.069918 seconds
minimize_ executed in 22.683234 seconds
minimize_ executed in 23.550765 seconds
minimize_ executed in 27.105877 seconds
minimize_ executed in 25.571651 seconds
compare_on executed in 153.404157 seconds
Generational Distance (GD): (0.004089128641172405, 0.002410436833815593) | (0.0039818584620647515, 0.0023971213502827025) | (0.003875556888587247, 0.0024870246383765344) | (0.004122672235573774, 0.0023888190880321453) | (0.004213267732671298, 0.0025000526580224317) | (0.003958823804447007, 0.0025021774702709823)
Generational Distance Plus (GD+): (0.00167304054567625, 0.0016132951799552736) | (0.001650639077822797, 0.001571810489816805) | (0.0016542729602016346, 0.0015606047760458042) | (0.0016696488936817837, 0.0015642043793963056) | (0.0016867080624874513, 0.00156368547764

In [15]:
pf, *results = test("carside", n_gen=500)
# pf, results[0].F, results[1].F, results[2].F

Problem <bound method Problem.name of <pymoo.problems.multi.carside.Carside object at 0x12427e5b0>>: n_var=7 n_obj=3 n_ieq_constr=10 n_eq_constr=0
minimize_ executed in 31.618004 seconds
minimize_ executed in 27.380150 seconds
minimize_ executed in 24.635842 seconds
minimize_ executed in 23.547156 seconds
minimize_ executed in 26.276621 seconds
minimize_ executed in 26.472269 seconds
compare_on executed in 159.932191 seconds
Generational Distance (GD): (1.06831029381358, 0.4374746997633727) | (1.0611268427504033, 0.4359107972230391) | (1.080625157726749, 0.4463358215866472) | (1.0865203305960534, 0.44506203612406714) | (1.0776230187017266, 0.42312226409771325) | (1.0776754513578477, 0.4365565825210747)
Generational Distance Plus (GD+): (0.5806094879349967, 0.0) | (0.6093094336819673, 0.0) | (0.6379240950352298, 0.0) | (0.6183424444177721, 0.0) | (0.6377530191001812, 0.0) | (0.6321535866531077, 0.0)
Inverted Generational Distance (IGD): (5.064402597121622, 4.604086846977223) | (5.037736

In [ ]:
pf, *results = test("g1", n_gen=10)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G1 object at 0x124620c10>>: n_var=13 n_obj=1 n_ieq_constr=9 n_eq_constr=0
minimize_ executed in 0.423980 seconds
minimize_ executed in 0.474960 seconds
minimize_ executed in 0.386205 seconds
minimize_ executed in 0.455116 seconds
minimize_ executed in 0.504107 seconds
minimize_ executed in 0.471798 seconds
compare_on executed in 2.717479 seconds
Generational Distance (GD): (7.804840529710274, 7.804840529710274) | (18.126566996724783, 18.126566996724783) | (18.286682079299737, 18.286682079299737) | (17.546958256118955, 17.546958256118955) | (23.46795517091956, 23.46795517091956) | (23.306308013159935, 23.306308013159935)
Generational Distance Plus (GD+): (7.804840529710274, 7.804840529710274) | (18.126566996724783, 18.126566996724783) | (18.286682079299737, 18.286682079299737) | (17.546958256118955, 17.546958256118955) | (23.46795517091956, 23.46795517091956) | (23.306308013159935, 23.306308013159935)
Inverted Generational D

(array([[-15.]]),
 array([-7.19515947]),
 array([[-4.82294585,  0.        ]]),
 array([[-4.54042346,  0.        ]]),
 array([[-5.89529001,  0.        ]]),
 array([[-4.96282311,  0.        ,  0.        ]]),
 array([[-5.34681435,  0.        ,  0.        ]]))

In [11]:
pf, *results = test("g1", n_gen=20)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G1 object at 0x123bd1910>>: n_var=13 n_obj=1 n_ieq_constr=9 n_eq_constr=0
minimize_ executed in 1.101835 seconds
minimize_ executed in 1.093827 seconds


KeyboardInterrupt: 

In [ ]:
pf, *results = test("g1", n_gen=50)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G1 object at 0x121e29d00>>: n_var=13 n_obj=1 n_ieq_constr=9 n_eq_constr=0
minimize_ executed in 3.962497 seconds
minimize_ executed in 3.890079 seconds
minimize_ executed in 3.741259 seconds
minimize_ executed in 3.219460 seconds
minimize_ executed in 3.585010 seconds
minimize_ executed in 3.997255 seconds
compare_on executed in 22.397259 seconds
Generational Distance (GD): (0.23195412632949797, 0.23195412632949797) | (15.006402805722532, 15.006402805722532) | (15.008243727181354, 15.008243727181354) | (15.00398549460083, 15.00398549460083) | (21.217035957909207, 21.217035957909207) | (21.21583455102328, 21.21583455102328)
Generational Distance Plus (GD+): (0.23195412632949797, 0.23195412632949797) | (15.006402805722532, 15.006402805722532) | (15.008243727181354, 15.008243727181354) | (15.00398549460083, 15.00398549460083) | (21.217035957909207, 21.217035957909207) | (21.21583455102328, 21.21583455102328)
Inverted Generatio

(array([[-15.]]),
 array([-14.76804587]),
 array([[-14.56167915,   0.        ]]),
 array([[-14.50262713,   0.        ]]),
 array([[-14.65419554,   0.        ]]),
 array([[-14.5967447,   0.       ,   0.       ]]),
 array([[-14.66588072,   0.        ,   0.        ]]))

In [ ]:
pf, *results = test("g1", n_gen=100)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G1 object at 0x1217049d0>>: n_var=13 n_obj=1 n_ieq_constr=9 n_eq_constr=0
minimize_ executed in 8.122535 seconds


KeyboardInterrupt: 

In [ ]:
pf, *results = test("g1", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G1 object at 0x11a7bbfa0>>: n_var=13 n_obj=1 n_ieq_constr=9 n_eq_constr=0
minimize_ executed in 17.621662 seconds
minimize_ executed in 17.115309 seconds
minimize_ executed in 16.050240 seconds
minimize_ executed in 15.222720 seconds
minimize_ executed in 17.552674 seconds
minimize_ executed in 17.301050 seconds
compare_on executed in 100.864651 seconds
Generational Distance (GD): (0.0012008802103107286, 0.0012008802103107286) | (15.00000213209419, 15.00000213209419) | (15.000000002238142, 15.000000002238142) | (15.00000242786562, 15.00000242786562) | (21.213203435671016, 21.213203435671016) | (21.21320389828282, 21.21320389828282)
Generational Distance Plus (GD+): (0.0012008802103107286, 0.0012008802103107286) | (15.00000213209419, 15.00000213209419) | (15.000000002238142, 15.000000002238142) | (15.00000242786562, 15.00000242786562) | (21.213203435671016, 21.213203435671016) | (21.21320389828282, 21.21320389828282)
Inverte

(array([[-15.]]),
 array([-14.99879912]),
 array([[-14.99200232,   0.        ]]),
 array([[-14.99974088,   0.        ]]),
 array([[-14.9914656,   0.       ]]),
 array([[-14.99994375,   0.        ,   0.        ]]),
 array([[-14.99556941,   0.        ,   0.        ]]))

In [ ]:
pf, *results = test("g2", n_gen=10)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G2 object at 0x120038b20>>: n_var=20 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 0.969674 seconds
minimize_ executed in 0.789556 seconds
minimize_ executed in 0.692098 seconds
minimize_ executed in 0.805410 seconds
minimize_ executed in 0.781961 seconds
minimize_ executed in 0.780550 seconds
compare_on executed in 4.821200 seconds
Generational Distance (GD): (0.383150337117087, 0.383150337117087) | (0.8902852606600589, 0.8902852606600589) | (0.8902852606600589, 0.8902852606600589) | (0.8902852606600589, 0.8902852606600589) | (1.1993379464788732, 1.1993379464788732) | (1.1993379464788732, 1.1993379464788732)
Generational Distance Plus (GD+): (0.383150337117087, 0.383150337117087) | (0.8902852606600589, 0.8902852606600589) | (0.8902852606600589, 0.8902852606600589) | (0.8902852606600589, 0.8902852606600589) | (1.1993379464788732, 1.1993379464788732) | (1.1993379464788732, 1.1993379464788732)
Inverted Generation

(array([[-0.8036191]]),
 array([-0.42046877]),
 array([[-0.42046877,  0.        ]]),
 array([[-0.42046877,  0.        ]]),
 array([[-0.42046877,  0.        ]]),
 array([[-0.42046877,  0.        ,  0.        ]]),
 array([[-0.42046877,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g2", n_gen=50)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G2 object at 0x128cc30a0>>: n_var=20 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 4.616302 seconds
minimize_ executed in 3.883868 seconds
minimize_ executed in 3.647440 seconds
minimize_ executed in 3.730110 seconds
minimize_ executed in 3.870905 seconds
minimize_ executed in 3.739267 seconds
compare_on executed in 23.490646 seconds
Generational Distance (GD): (0.0234557404324518, 0.0234557404324518) | (0.803961340037471, 0.803961340037471) | (0.803961340037471, 0.803961340037471) | (0.803961340037471, 0.803961340037471) | (1.1367310591298443, 1.1367310591298443) | (1.1367310591298443, 1.1367310591298443)
Generational Distance Plus (GD+): (0.0234557404324518, 0.0234557404324518) | (0.803961340037471, 0.803961340037471) | (0.803961340037471, 0.803961340037471) | (0.803961340037471, 0.803961340037471) | (1.1367310591298443, 1.1367310591298443) | (1.1367310591298443, 1.1367310591298443)
Inverted Generational Dist

(array([[-0.8036191]]),
 array([-0.78016336]),
 array([[-0.78016336,  0.        ]]),
 array([[-0.78016336,  0.        ]]),
 array([[-0.78016336,  0.        ]]),
 array([[-0.78016336,  0.        ,  0.        ]]),
 array([[-0.78016336,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g2", n_gen=100)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G2 object at 0x1245df3a0>>: n_var=20 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 6.022397 seconds
minimize_ executed in 5.280904 seconds
minimize_ executed in 4.955304 seconds
minimize_ executed in 5.062992 seconds
minimize_ executed in 5.412051 seconds
minimize_ executed in 5.430928 seconds
compare_on executed in 32.166115 seconds
Generational Distance (GD): (0.009550922349130242, 0.009550922349130242) | (0.8036758579385912, 0.8036758579385912) | (0.8036758579385912, 0.8036758579385912) | (0.8036758579385912, 0.8036758579385912) | (1.136529167751071, 1.136529167751071) | (1.136529167751071, 1.136529167751071)
Generational Distance Plus (GD+): (0.009550922349130242, 0.009550922349130242) | (0.8036758579385912, 0.8036758579385912) | (0.8036758579385912, 0.8036758579385912) | (0.8036758579385912, 0.8036758579385912) | (1.136529167751071, 1.136529167751071) | (1.136529167751071, 1.136529167751071)
Inverted Gener

(array([[-0.8036191]]),
 array([-0.79406818]),
 array([[-0.79406818,  0.        ]]),
 array([[-0.79406818,  0.        ]]),
 array([[-0.79406818,  0.        ]]),
 array([[-0.79406818,  0.        ,  0.        ]]),
 array([[-0.79406818,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g2", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G2 object at 0x11a7bb6a0>>: n_var=20 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 18.530211 seconds
minimize_ executed in 16.377806 seconds
minimize_ executed in 14.916778 seconds
minimize_ executed in 15.004437 seconds
minimize_ executed in 16.266184 seconds


/Users/macbook/Desktop/workspace/pymoo-experiments/pymoo/constraints/as_obj.py:292: RuntimeWarning: invalid value encountered in divide
  wcvf *= (1 - constraint_violation_frequencies /


minimize_ executed in 15.943351 seconds
compare_on executed in 97.040850 seconds
Generational Distance (GD): (0.00389620967775961, 0.00389620967775961) | (0.8036285491229543, 0.8036285491229543) | (0.8036285491229543, 0.8036285491229543) | (0.8036285491229543, 0.8036285491229543) | (1.136495714677832, 1.136495714677832) | (1.136495714677832, 1.136495714677832)
Generational Distance Plus (GD+): (0.00389620967775961, 0.00389620967775961) | (0.8036285491229543, 0.8036285491229543) | (0.8036285491229543, 0.8036285491229543) | (0.8036285491229543, 0.8036285491229543) | (1.136495714677832, 1.136495714677832) | (1.136495714677832, 1.136495714677832)
Inverted Generational Distance (IGD): (0.00389620967775961, 0.00389620967775961) | (0.8036285491229543, 0.8036285491229543) | (0.8036285491229543, 0.8036285491229543) | (0.8036285491229543, 0.8036285491229543) | (1.136495714677832, 1.136495714677832) | (1.136495714677832, 1.136495714677832)
Inverted Generational Distance Plus (IGD+): (0.0038962096

(array([[-0.8036191]]),
 array([-0.79972289]),
 array([[-0.79972289,  0.        ]]),
 array([[-0.79972289,  0.        ]]),
 array([[-0.79972289,  0.        ]]),
 array([[-0.79972289,  0.        ,  0.        ]]),
 array([[-0.79972289,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g3", n_gen=10)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G3 object at 0x1293b9a00>>: n_var=10 n_obj=1 n_ieq_constr=0 n_eq_constr=1
minimize_ executed in 0.895306 seconds
minimize_ executed in 0.609716 seconds
minimize_ executed in 0.622958 seconds
minimize_ executed in 0.738658 seconds
minimize_ executed in 0.837687 seconds
minimize_ executed in 0.563300 seconds
compare_on executed in 4.269668 seconds
algo 0 not converge
Performance indicators could not be calculated due to invalid data.


(array([[-1.]]),
 None,
 array([[-0.19289964,  0.        ]]),
 array([[-0.00331462,  0.        ]]),
 array([[-39823.40621706,      0.        ]]),
 array([[-0.03819896,  1.        ,  0.        ]]),
 array([[-0.19289964,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g3", n_gen=50)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G3 object at 0x12085cdc0>>: n_var=10 n_obj=1 n_ieq_constr=0 n_eq_constr=1
minimize_ executed in 3.253845 seconds
minimize_ executed in 2.952708 seconds
minimize_ executed in 2.959109 seconds
minimize_ executed in 3.474277 seconds
minimize_ executed in 2.918357 seconds
minimize_ executed in 2.858128 seconds
compare_on executed in 18.418289 seconds
Generational Distance (GD): (0.9649442526531863, 0.9649442526531863) | (1.1796881714179537, 1.1796881714179537) | (1.0028640462823448, 1.0028640462823448) | (99960.0558841093, 99960.0558841093) | (1.6165705260858498, 1.6165705260858498) | (1.5465006245661324, 1.5465006245661324)
Generational Distance Plus (GD+): (0.9649442526531863, 0.9649442526531863) | (1.1796881714179537, 1.1796881714179537) | (1.0028640462823448, 1.0028640462823448) | (1.0000000000000009, 1.0000000000000009) | (1.6165705260858498, 1.6165705260858498) | (1.5465006245661324, 1.5465006245661324)
Inverted Generatio

(array([[-1.]]),
 array([-0.03505575]),
 array([[-0.37416921,  0.        ]]),
 array([[-0.92426167,  0.        ]]),
 array([[-99961.05587911,      0.        ]]),
 array([[-0.2168651,  0.       ,  0.       ]]),
 array([[-0.37416921,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g3", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G3 object at 0x11a8ee550>>: n_var=10 n_obj=1 n_ieq_constr=0 n_eq_constr=1
minimize_ executed in 15.505913 seconds
minimize_ executed in 15.357085 seconds
minimize_ executed in 14.595480 seconds
minimize_ executed in 21.690431 seconds
minimize_ executed in 13.943795 seconds
minimize_ executed in 14.220160 seconds
compare_on executed in 95.314045 seconds
Generational Distance (GD): (0.9332838223532404, 0.9332838223532404) | (1.0514205878734022, 1.0514205878734022) | (1.000001838827649, 1.000001838827649) | (99999.00000500011, 99999.00000500011) | (2.2597973429861913, 2.2597973429861913) | (1.4510290323091584, 1.4510290323091584)
Generational Distance Plus (GD+): (0.9332838223532404, 0.9332838223532404) | (1.0514205878734022, 1.0514205878734022) | (1.000001838827649, 1.000001838827649) | (1.0000000000000009, 1.0000000000000009) | (2.2597973429861913, 2.2597973429861913) | (1.4510290323091584, 1.4510290323091584)
Inverted Gener

(array([[-1.]]),
 array([-0.06671618]),
 array([[-0.67521507,  0.        ]]),
 array([[-0.99808228,  0.        ]]),
 array([[-100000.,       0.]]),
 array([[-0.67337478,  1.        ,  0.        ],
        [-0.66619266,  0.        ,  0.        ]]),
 array([[-0.67521507,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g3", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G3 object at 0x11a40ea90>>: n_var=10 n_obj=1 n_ieq_constr=0 n_eq_constr=1
minimize_ executed in 51.936240 seconds
minimize_ executed in 42.992641 seconds
minimize_ executed in 41.471907 seconds
minimize_ executed in 79.972935 seconds
minimize_ executed in 49.231563 seconds
minimize_ executed in 49.154411 seconds
compare_on executed in 314.761320 seconds
Generational Distance (GD): (0.3998351711696747, 0.3998351711696747) | (1.000826689550189, 1.000826689550189) | (1.0000003428252404, 1.0000003428252404) | (99999.00000500011, 99999.00000500011) | (2.2379821615199442, 2.2379821615199442) | (1.4147982409220021, 1.4147982409220021)
Generational Distance Plus (GD+): (0.3998351711696747, 0.3998351711696747) | (1.000826689550189, 1.000826689550189) | (1.0000003428252404, 1.0000003428252404) | (1.0000000000000009, 1.0000000000000009) | (2.2379821615199442, 2.2379821615199442) | (1.4147982409220021, 1.4147982409220021)
Inverted Gene

(array([[-1.]]),
 array([-0.60016483]),
 array([[-0.95932983,  0.        ]]),
 array([[-0.99917196,  0.        ]]),
 array([[-100000.,       0.]]),
 array([[-0.90745728,  1.        ,  0.        ],
        [-0.89917152,  0.        ,  0.        ]]),
 array([[-0.95932983,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g4", n_gen=10)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G4 object at 0x128bb4460>>: n_var=5 n_obj=1 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 0.730644 seconds
minimize_ executed in 0.666064 seconds
minimize_ executed in 0.625428 seconds
minimize_ executed in 0.565959 seconds
minimize_ executed in 0.661558 seconds
minimize_ executed in 0.713101 seconds
compare_on executed in 3.964383 seconds
Generational Distance (GD): (325.7438343148606, 325.7438343148606) | (30668.11850266796, 30668.11850266796) | (30668.623125550854, 30668.623125550854) | (30667.10282149713, 30667.10282149713) | (43369.14921300867, 43369.14921300867) | (43369.218633940756, 43369.218633940756)
Generational Distance Plus (GD+): (325.7438343148606, 325.7438343148606) | (30668.11850266796, 30668.11850266796) | (30668.623125550854, 30668.623125550854) | (30667.10282149713, 30667.10282149713) | (43369.14921300867, 43369.14921300867) | (43369.218633940756, 43369.218633940756)
Inverted Generational Distance (

(array([[-30665.53867178]]),
 array([-30339.79483747]),
 array([[-30267.75674277,      0.        ]]),
 array([[-30230.58765565,      0.        ]]),
 array([[-30355.80744372,      0.        ]]),
 array([[-30301.42413083,      0.        ,      0.        ]]),
 array([[-30293.24731105,      0.        ,      0.        ]]))

In [ ]:
pf, *results = test("g4", n_gen=50)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G4 object at 0x12902e1c0>>: n_var=5 n_obj=1 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 4.418395 seconds
minimize_ executed in 3.509815 seconds
minimize_ executed in 3.057084 seconds
minimize_ executed in 3.085104 seconds
minimize_ executed in 3.422367 seconds
minimize_ executed in 3.411980 seconds
compare_on executed in 20.906359 seconds
Generational Distance (GD): (12.664750541709509, 12.664750541709509) | (30665.54876240654, 30665.54876240654) | (30665.538939221053, 30665.538939221053) | (30665.573339387658, 30665.573339387658) | (43367.63081907145, 43367.63081907145) | (43367.622550029526, 43367.622550029526)
Generational Distance Plus (GD+): (12.664750541709509, 12.664750541709509) | (30665.54876240654, 30665.54876240654) | (30665.538939221053, 30665.538939221053) | (30665.573339387658, 30665.573339387658) | (43367.63081907145, 43367.63081907145) | (43367.622550029526, 43367.622550029526)
Inverted Generational D

(array([[-30665.53867178]]),
 array([-30652.87392124]),
 array([[-30640.66159609,      0.        ]]),
 array([[-30661.48870332,      0.        ]]),
 array([[-30619.42790596,      0.        ]]),
 array([[-30635.89414447,      0.        ,      0.        ]]),
 array([[-30652.82723783,      0.        ,      0.        ]]))

In [ ]:
pf, *results = test("g4", n_gen=100)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G4 object at 0x12895f340>>: n_var=5 n_obj=1 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 7.529602 seconds
minimize_ executed in 6.621638 seconds
minimize_ executed in 6.166270 seconds
minimize_ executed in 6.206654 seconds
minimize_ executed in 6.819944 seconds
minimize_ executed in 6.777514 seconds
compare_on executed in 40.123183 seconds
Generational Distance (GD): (12.33777785107668, 12.33777785107668) | (30665.5392497039, 30665.5392497039) | (30665.538902879955, 30665.538902879955) | (30665.53995663075, 30665.53995663075) | (43367.62100223213, 43367.62100223213) | (43367.622252131194, 43367.622252131194)
Generational Distance Plus (GD+): (12.33777785107668, 12.33777785107668) | (30665.5392497039, 30665.5392497039) | (30665.538902879955, 30665.538902879955) | (30665.53995663075, 30665.53995663075) | (43367.62100223213, 43367.62100223213) | (43367.622252131194, 43367.622252131194)
Inverted Generational Distance (IGD

(array([[-30665.53867178]]),
 array([-30653.20089393]),
 array([[-30659.58514407,      0.        ]]),
 array([[-30661.77391337,      0.        ]]),
 array([[-30656.6616746,      0.       ]]),
 array([[-30660.31067318,      0.        ,      0.        ],
        [-30660.31067318,      0.        ,      0.        ]]),
 array([[-30653.88782339,      0.        ,      0.        ]]))

In [ ]:
pf, *results = test("g4", n_gen=200)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G4 object at 0x128fefc10>>: n_var=5 n_obj=1 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 18.003653 seconds
minimize_ executed in 15.140371 seconds
minimize_ executed in 15.194666 seconds
minimize_ executed in 13.049798 seconds
minimize_ executed in 13.730051 seconds
minimize_ executed in 13.283730 seconds
compare_on executed in 88.404588 seconds
Generational Distance (GD): (1.9788551475285203, 1.9788551475285203) | (30665.53867294197, 30665.53867294197) | (30665.538902383814, 30665.538902383814) | (30665.539922298056, 30665.539922298056) | (43367.621001873806, 43367.621001873806) | (43367.62225211823, 43367.62225211823)
Generational Distance Plus (GD+): (1.9788551475285203, 1.9788551475285203) | (30665.53867294197, 30665.53867294197) | (30665.538902383814, 30665.538902383814) | (30665.539922298056, 30665.539922298056) | (43367.621001873806, 43367.621001873806) | (43367.62225211823, 43367.62225211823)
Inverted Generati

(array([[-30665.53867178]]),
 array([-30663.55981664]),
 array([[-30665.27209845,      0.        ]]),
 array([[-30661.77795682,      0.        ]]),
 array([[-30656.78107978,      0.        ]]),
 array([[-30660.31364643,      0.        ,      0.        ],
        [-30660.31364643,      0.        ,      0.        ],
        [-30660.31364643,      0.        ,      0.        ]]),
 array([[-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-30653.88787164,      0.        ,      0.        ],
        [-3

In [ ]:
pf, *results = test("g4", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G4 object at 0x123d08880>>: n_var=5 n_obj=1 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 18.749425 seconds
minimize_ executed in 16.849741 seconds
minimize_ executed in 15.789038 seconds
minimize_ executed in 15.258252 seconds
minimize_ executed in 17.584501 seconds
minimize_ executed in 16.954024 seconds
compare_on executed in 101.185987 seconds
Generational Distance (GD): (0.8562657066213433, 0.8562657066213433) | (30665.538672941904, 30665.538672941904) | (30665.538902383814, 30665.538902383814) | (30665.53868585182, 30665.53868585182) | (43367.621001873806, 43367.621001873806) | (43367.62225211823, 43367.62225211823)
Generational Distance Plus (GD+): (0.8562657066213433, 0.8562657066213433) | (30665.538672941904, 30665.538672941904) | (30665.538902383814, 30665.538902383814) | (30665.53868585182, 30665.53868585182) | (43367.621001873806, 43367.621001873806) | (43367.62225211823, 43367.62225211823)
Inverted Generat

(array([[-30665.53867178]]),
 array([-30664.68240608]),
 array([[-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.27210584,      0.        ],
        [-30665.2721

In [ ]:
pf, *results = test("g5", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G5 object at 0x11af64c70>>: n_var=4 n_obj=1 n_ieq_constr=2 n_eq_constr=3
minimize_ executed in 15.778730 seconds
minimize_ executed in 15.623043 seconds
minimize_ executed in 13.911150 seconds
minimize_ executed in 14.496536 seconds
minimize_ executed in 13.378300 seconds
minimize_ executed in 13.103845 seconds
compare_on executed in 86.292667 seconds
algo 0 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[5126.4981096]]),
 None,
 array([[5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ]]),
 array([[5207.74647402,    0.        ]]),
 array([[5157.16929248,    0.        ]]),
 None,
 None)

In [ ]:
pf, *results = test("g5", n_gen=500)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G5 object at 0x1249c5040>>: n_var=4 n_obj=1 n_ieq_constr=2 n_eq_constr=3
minimize_ executed in 26.689639 seconds
minimize_ executed in 27.122147 seconds
minimize_ executed in 22.947722 seconds
minimize_ executed in 24.381045 seconds
minimize_ executed in 23.405280 seconds
minimize_ executed in 22.660087 seconds
compare_on executed in 147.207344 seconds
algo 0 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[5126.4981096]]),
 None,
 array([[5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
        [5159.58555777,    0.        ],
       

In [ ]:
pf, *results = test("g6", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G6 object at 0x123bbc520>>: n_var=2 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 19.377441 seconds
minimize_ executed in 16.063445 seconds
minimize_ executed in 14.022820 seconds
minimize_ executed in 14.211249 seconds
minimize_ executed in 16.599536 seconds
minimize_ executed in 17.040343 seconds
compare_on executed in 97.316968 seconds
Generational Distance (GD): (75.45310435168813, 75.45310435168813) | (6962.019202251239, 6962.019202251239) | (6961.954921355926, 6961.954921355926) | (6962.10272678499, 6962.10272678499) | (9845.506441701646, 9845.506441701646) | (9845.80062853557, 9845.80062853557)
Generational Distance Plus (GD+): (75.45310435168813, 75.45310435168813) | (6962.019202251239, 6962.019202251239) | (6961.954921355926, 6961.954921355926) | (6962.10272678499, 6962.10272678499) | (9845.506441701646, 9845.506441701646) | (9845.80062853557, 9845.80062853557)
Inverted Generational Distance (IGD): (75

(array([[-6961.81387558]]),
 array([-6886.36077123]),
 array([[-6908.34487246,     0.        ]]),
 array([[-6917.4980909,     0.       ]]),
 array([[-6898.39510796,     0.        ]]),
 array([[-6944.71951545,     0.        ,     0.        ]]),
 array([[-6883.80656547,     0.        ,     0.        ]]))

In [ ]:
pf, *results = test("g7", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G7 object at 0x11af3dc10>>: n_var=10 n_obj=1 n_ieq_constr=8 n_eq_constr=0
minimize_ executed in 17.779341 seconds
minimize_ executed in 14.752716 seconds
minimize_ executed in 13.597697 seconds
minimize_ executed in 15.025621 seconds
minimize_ executed in 16.848142 seconds
minimize_ executed in 17.548712 seconds
compare_on executed in 95.554477 seconds
Generational Distance (GD): (0.23208261557518384, 0.23208261557518384) | (24.347779363111975, 24.347779363111975) | (24.311083186329277, 24.311083186329277) | (24.30984484443571, 24.30984484443571) | (34.38071534788415, 34.38071534788415) | (34.375669597941354, 34.375669597941354)
Generational Distance Plus (GD+): (0.23208261557518384, 0.23208261557518384) | (1.4221675753746155, 1.4221675753746155) | (0.4867919373690981, 0.4867919373690981) | (0.4204248544066864, 0.4204248544066864) | (0.6708123639048686, 0.6708123639048686) | (0.32103224473596725, 0.32103224473596725)
Invert

(array([[24.30620907]]),
 array([24.53829168]),
 array([[25.72837664,  0.        ]]),
 array([[24.79300101,  0.        ]]),
 array([[24.72663392,  0.        ]]),
 array([[24.97702143,  0.        ,  0.        ]]),
 array([[24.62724131,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g8", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G8 object at 0x11a656a00>>: n_var=2 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 18.343232 seconds
minimize_ executed in 15.082446 seconds
minimize_ executed in 13.897455 seconds
minimize_ executed in 13.875354 seconds
minimize_ executed in 16.258907 seconds
minimize_ executed in 16.799631 seconds
compare_on executed in 94.258236 seconds
Generational Distance (GD): (7.296167836567946e-11, 7.296167836567946e-11) | (0.09582504141803586, 0.09582504141803586) | (0.09582504141803586, 0.09582504141803586) | (0.09582504141803586, 0.09582504141803586) | (0.13551707318834988, 0.13551707318834988) | (0.13551707318834988, 0.13551707318834988)
Generational Distance Plus (GD+): (7.296167836567946e-11, 7.296167836567946e-11) | (0.09582504141803586, 0.09582504141803586) | (0.09582504141803586, 0.09582504141803586) | (0.09582504141803586, 0.09582504141803586) | (0.13551707318834988, 0.13551707318834988) | (0.13551707318834988

(array([[-0.09582504]]),
 array([[-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [-0.09582504],
        [

In [ ]:
pf, *results = test("g9", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G9 object at 0x11a780c10>>: n_var=7 n_obj=1 n_ieq_constr=4 n_eq_constr=0
minimize_ executed in 16.315844 seconds
minimize_ executed in 14.190745 seconds
minimize_ executed in 14.245097 seconds
minimize_ executed in 13.118379 seconds
minimize_ executed in 13.982106 seconds
minimize_ executed in 14.732587 seconds
compare_on executed in 86.586453 seconds
Generational Distance (GD): (0.04538406001972817, 0.04538406001972817) | (680.6300633488488, 680.6300633488488) | (680.6300806743315, 680.6300806743315) | (680.6300835634632, 680.6300835634632) | (962.5563897483369, 962.5563897483369) | (962.5563489957727, 962.5563489957727)
Generational Distance Plus (GD+): (0.04538406001972817, 0.04538406001972817) | (0.09018188435481989, 0.09018188435481989) | (0.17809340640235405, 0.17809340640235405) | (0.18881239716449727, 0.18881239716449727) | (0.5034306118674294, 0.5034306118674294) | (0.41831698563680675, 0.41831698563680675)
Inverte

(array([[680.63005737]]),
 array([680.67544143]),
 array([[680.72023926,   0.        ]]),
 array([[680.80815078,   0.        ]]),
 array([[680.81886977,   0.        ]]),
 array([[681.13348799,   0.        ,   0.        ]]),
 array([[681.04837436,   0.        ,   0.        ]]))

In [ ]:
pf, *results = test("g10", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G10 object at 0x11aabd970>>: n_var=8 n_obj=1 n_ieq_constr=6 n_eq_constr=0
minimize_ executed in 19.817633 seconds
minimize_ executed in 16.044791 seconds
minimize_ executed in 11.451505 seconds
minimize_ executed in 12.470692 seconds
minimize_ executed in 15.869799 seconds
minimize_ executed in 14.257913 seconds
compare_on executed in 89.914020 seconds
algo 2 not converge
algo 3 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[7049.24802181]]),
 array([7558.3840745]),
 array([[7523.67785762,    0.        ]]),
 None,
 None,
 array([[7868.72503782,    0.        ,    0.        ]]),
 None)

In [ ]:
pf, *results = test("g11", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G11 object at 0x11acd8430>>: n_var=2 n_obj=1 n_ieq_constr=1 n_eq_constr=0
minimize_ executed in 17.286893 seconds
minimize_ executed in 15.814525 seconds
minimize_ executed in 14.102475 seconds
minimize_ executed in 14.830634 seconds
minimize_ executed in 15.890005 seconds
minimize_ executed in 16.187569 seconds
compare_on executed in 94.113631 seconds
Generational Distance (GD): (6.016776429307313e-07, 6.016776429307313e-07) | (0.7500000000806363, 0.7500000000806363) | (0.7500000002469247, 0.7500000002469247) | (0.7500000002469247, 0.7500000002469247) | (1.06066017183684, 1.06066017183684) | (1.06066017183684, 1.06066017183684)
Generational Distance Plus (GD+): (6.016776429307313e-07, 6.016776429307313e-07) | (1.0997918215793767e-05, 1.0997918215793767e-05) | (1.9245435417092338e-05, 1.9245435417092338e-05) | (1.9245435417092338e-05, 1.9245435417092338e-05) | (1.0997918215793767e-05, 1.0997918215793767e-05) | (1.0997918215

(array([[0.75]]),
 array([0.7500006]),
 array([[0.750011, 0.      ]]),
 array([[0.75001925, 0.        ]]),
 array([[0.75001925, 0.        ]]),
 array([[0.750011, 0.      , 0.      ]]),
 array([[0.750011, 0.      , 0.      ]]))

In [ ]:
pf, *results = test("g12", n_gen=10)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G12 object at 0x12031df70>>: n_var=3 n_obj=1 n_ieq_constr=1 n_eq_constr=0
minimize_ executed in 0.935786 seconds
minimize_ executed in 0.744768 seconds
minimize_ executed in 0.705743 seconds
minimize_ executed in 0.734979 seconds
minimize_ executed in 0.764109 seconds
minimize_ executed in 0.741152 seconds
compare_on executed in 4.628503 seconds
Generational Distance (GD): (1.3290374173080544e-05, 1.3290374173080544e-05) | (1.0000000026988347, 1.0000000026988347) | (1.0000000001649383, 1.0000000001649383) | (1.0000000013787564, 1.0000000013787564) | (1.4142135642814595, 1.4142135642814595) | (1.4142135642814595, 1.4142135642814595)
Generational Distance Plus (GD+): (1.3290374173080544e-05, 1.3290374173080544e-05) | (1.0000000026988347, 1.0000000026988347) | (1.0000000001649383, 1.0000000001649383) | (1.0000000013787564, 1.0000000013787564) | (1.4142135642814595, 1.4142135642814595) | (1.4142135642814595, 1.4142135642814595)

(array([[-1.]]),
 array([-0.99998671]),
 array([[-0.99992653,  0.        ]]),
 array([[-0.99998184,  0.        ]]),
 array([[-0.99994749,  0.        ]]),
 array([[-0.99992653,  0.        ,  0.        ]]),
 array([[-0.99992653,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g12", n_gen=50)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G12 object at 0x11a60c8b0>>: n_var=3 n_obj=1 n_ieq_constr=1 n_eq_constr=0
minimize_ executed in 3.226351 seconds
minimize_ executed in 2.930271 seconds
minimize_ executed in 2.684064 seconds
minimize_ executed in 2.802008 seconds
minimize_ executed in 2.868441 seconds
minimize_ executed in 2.935085 seconds
compare_on executed in 17.447626 seconds
Generational Distance (GD): (4.042935541903603e-09, 4.042935541903603e-09) | (1.0, 1.0) | (1.0, 1.0) | (1.0, 1.0) | (1.4142135623730951, 1.4142135623730951) | (1.4142135623730951, 1.4142135623730951)
Generational Distance Plus (GD+): (4.042935541903603e-09, 4.042935541903603e-09) | (1.0, 1.0) | (1.0, 1.0) | (1.0, 1.0) | (1.4142135623730951, 1.4142135623730951) | (1.4142135623730951, 1.4142135623730951)
Inverted Generational Distance (IGD): (4.042935541903603e-09, 4.042935541903603e-09) | (1.0, 1.0) | (1.0, 1.0) | (1.0, 1.0) | (1.4142135623730951, 1.4142135623730951) | (1.4142135623

(array([[-1.]]),
 array([-1.]),
 array([[-1.,  0.]]),
 array([[-1.,  0.]]),
 array([[-1.,  0.]]),
 array([[-1.,  0.,  0.]]),
 array([[-1.,  0.,  0.]]))

In [ ]:
pf, *results = test("g12", n_gen=100)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G12 object at 0x11a7f9bb0>>: n_var=3 n_obj=1 n_ieq_constr=1 n_eq_constr=0
minimize_ executed in 6.709824 seconds
minimize_ executed in 6.102712 seconds
minimize_ executed in 5.739135 seconds
minimize_ executed in 5.905752 seconds
minimize_ executed in 7.005141 seconds
minimize_ executed in 6.521468 seconds
compare_on executed in 37.985266 seconds
Generational Distance (GD): (3.9455994027548513e-10, 3.9455994027548513e-10) | (1.0, 1.0) | (1.0, 1.0) | (1.0, 1.0) | (1.4142135623730951, 1.4142135623730951) | (1.4142135623730951, 1.4142135623730951)
Generational Distance Plus (GD+): (3.9455994027548513e-10, 3.9455994027548513e-10) | (1.0, 1.0) | (1.0, 1.0) | (1.0, 1.0) | (1.4142135623730951, 1.4142135623730951) | (1.4142135623730951, 1.4142135623730951)
Inverted Generational Distance (IGD): (3.9455994027548513e-10, 3.9455994027548513e-10) | (1.0, 1.0) | (1.0, 1.0) | (1.0, 1.0) | (1.4142135623730951, 1.4142135623730951) | (1.4142

(array([[-1.]]),
 array([-1.]),
 array([[-1.,  0.],
        [-1.,  0.]]),
 array([[-1.,  0.]]))

In [ ]:
pf, *results = test("g13", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G13 object at 0x11a60cc40>>: n_var=5 n_obj=1 n_ieq_constr=0 n_eq_constr=3
minimize_ executed in 17.145337 seconds
minimize_ executed in 15.782083 seconds
minimize_ executed in 14.530997 seconds
minimize_ executed in 13.731573 seconds
minimize_ executed in 13.794807 seconds
minimize_ executed in 12.850622 seconds
compare_on executed in 87.836383 seconds
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.05394984]]),
 array([0.99272692]),
 array([[0.99213073, 0.        ]]),
 array([[0.00350376, 0.        ]]),
 array([[0.00349911, 0.        ]]),
 array([[0.46394303, 0.        , 0.        ],
        [0.46393916, 1.        , 0.        ]]),
 None)

In [ ]:
pf, *results = test("g14", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G14 object at 0x123f8f700>>: n_var=10 n_obj=1 n_ieq_constr=0 n_eq_constr=3
minimize_ executed in 15.423612 seconds
minimize_ executed in 13.925714 seconds
minimize_ executed in 10.651796 seconds
minimize_ executed in 10.519839 seconds
minimize_ executed in 15.744890 seconds
minimize_ executed in 13.893635 seconds
compare_on executed in 80.160735 seconds
algo 2 not converge
algo 3 not converge
Performance indicators could not be calculated due to invalid data.


(array([[-47.76488846]]),
 array([-39.36306116]),
 array([[-45.7644784,   0.       ]]),
 None,
 None,
 array([[-41.5749409 ,   2.        ,   0.        ],
        [-41.57396883,   1.        ,   0.        ],
        [-41.57576247,   3.        ,   0.        ],
        [-41.57246712,   0.        ,   0.        ]]),
 array([[-41.30285264,   0.52305476,   0.        ],
        [-41.30287862,   0.54047323,   0.        ],
        [-41.30436572,   1.16030534,   0.        ],
        [-41.30307573,   0.54135338,   0.        ],
        [-41.30425779,   1.15769712,   0.        ],
        [-41.30456335,   1.17852684,   0.        ],
        [-41.30310466,   0.57901391,   0.        ],
        [-41.30456487,   1.17997616,   0.        ],
        [-41.30449278,   1.16606061,   0.        ],
        [-41.30310438,   0.57888199,   0.        ],
        [-41.30578789,   2.        ,   0.        ],
        [-41.3041917 ,   0.59147026,   0.        ],
        [-41.30508226,   1.1875    ,   0.        ],
        [-41

In [ ]:
pf, *results = test("g15", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G15 object at 0x1242e81c0>>: n_var=3 n_obj=1 n_ieq_constr=0 n_eq_constr=2
minimize_ executed in 16.125552 seconds
minimize_ executed in 14.427292 seconds
minimize_ executed in 14.240602 seconds
minimize_ executed in 14.045748 seconds
minimize_ executed in 15.769687 seconds
minimize_ executed in 14.296551 seconds
compare_on executed in 88.906511 seconds
Generational Distance (GD): (5.197492997029144, 5.197492997029144) | (961.7150222914416, 961.7150222914416) | (961.7150225853828, 961.7150225853828) | (961.7150227840347, 961.7150227840347) | (1360.0704276840595, 1360.0704276840595) | (1359.784995998778, 1359.784995998778)
Generational Distance Plus (GD+): (5.197492997029144, 5.197492997029144) | (0.0016876554436748847, 0.0016876554436748847) | (0.02383743915629566, 0.02383743915629566) | (0.030827201249280733, 0.030827201249280733) | (0.008021254134291667, 0.008021254134291667) | (0.0017799373201796698, 0.0017799373201796698

(array([[961.71502229]]),
 array([[966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529],
        [966.91251529]]),
 array([[961.71670995,   0.        ]]),
 array([[961.73885973,   0.        ]]),
 array([[961.74584949,   0.        ],
        [961.74584949,   0.        ],
        [961.74584949,   0.        ]]),
 array([[961.72304354,   0.        ,   0.        ],
        [961.72298815,   1.        ,   0.        ],
        [961.72289661,   2.        ,   0.        ]]),
 array([[9.61716802e+02, 4.03703704e-01, 0.00000000e+00],
        [9.61716803e+02, 3.98809524e-01, 0.00000000e+00],
        [9.61716804e+02, 3.97205589e-01, 0.00000000e+00],
        [9.61716799e+02, 4.07331976e-01, 0.00000000e+00],
        [9.61716797e+02, 4.17293233e-01, 0.00000000e+00],
        [9.61716797e+02, 4.43396226e-01, 0.00000000e+00],
        [9.61716707e+02, 5.221

In [ ]:
pf, *results = test("g16", n_gen=5)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x12a4fce50>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 0.345116 seconds
minimize_ executed in 0.268288 seconds
minimize_ executed in 0.293867 seconds
minimize_ executed in 0.234030 seconds
minimize_ executed in 0.281810 seconds
minimize_ executed in 0.306334 seconds
compare_on executed in 1.731148 seconds
algo 4 not converge
Performance indicators could not be calculated due to invalid data.


(array([[-1.90515526]]),
 array([-1.46810563]),
 array([[-1.13305208,  0.        ]]),
 array([[-1.54785409,  0.        ]]),
 array([[-1.55266613,  0.        ]]),
 None,
 array([[-1.00666457,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=10)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x1210feca0>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 0.715130 seconds
minimize_ executed in 0.608831 seconds
minimize_ executed in 0.498417 seconds
minimize_ executed in 0.627144 seconds
minimize_ executed in 0.673576 seconds
minimize_ executed in 0.629694 seconds
compare_on executed in 3.755301 seconds
Generational Distance (GD): (0.3715071532989249, 0.3715071532989249) | (1.947022717997291, 1.947022717997291) | (1.9344350094997558, 1.9344350094997558) | (1.9316594602364816, 1.9316594602364816) | (2.704648401826814, 2.704648401826814) | (2.739864310326732, 2.739864310326732)
Generational Distance Plus (GD+): (0.3715071532989249, 0.3715071532989249) | (1.947022717997291, 1.947022717997291) | (1.9344350094997558, 1.9344350094997558) | (1.9316594602364816, 1.9316594602364816) | (2.704648401826814, 2.704648401826814) | (2.739864310326732, 2.739864310326732)
Inverted Generational Dist

(array([[-1.90515526]]),
 array([-1.5336481]),
 array([[-1.50355731,  0.        ]]),
 array([[-1.56986098,  0.        ]]),
 array([[-1.58626387,  0.        ]]),
 array([[-1.66874487,  0.        ,  0.        ]]),
 array([[-1.4075376,  0.       ,  0.       ]]))

In [ ]:
pf, *results = test("g16", n_gen=20)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x12146c820>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 1.548582 seconds
minimize_ executed in 1.237413 seconds
minimize_ executed in 1.128049 seconds
minimize_ executed in 1.164362 seconds
minimize_ executed in 1.203633 seconds
minimize_ executed in 1.307721 seconds
compare_on executed in 7.592369 seconds
Generational Distance (GD): (0.1986547387438593, 0.1986547387438593) | (1.9139716097419868, 1.9139716097419868) | (1.9073177778075743, 1.9073177778075743) | (1.9151338760422294, 1.9151338760422294) | (2.698185948213705, 2.698185948213705) | (2.699369083328074, 2.699369083328074)
Generational Distance Plus (GD+): (0.1986547387438593, 0.1986547387438593) | (1.9139716097419868, 1.9139716097419868) | (1.9073177778075743, 1.9073177778075743) | (1.9151338760422294, 1.9151338760422294) | (2.698185948213705, 2.698185948213705) | (2.699369083328074, 2.699369083328074)
Inverted Generational 

(array([[-1.90515526]]),
 array([-1.70650052]),
 array([[-1.72165929,  0.        ]]),
 array([[-1.81435575,  0.        ]]),
 array([[-1.70990872,  0.        ]]),
 array([[-1.76033018,  0.        ,  0.        ]]),
 array([[-1.73974565,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=50)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x121fd9790>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 3.920267 seconds
minimize_ executed in 3.417399 seconds
minimize_ executed in 2.932318 seconds
minimize_ executed in 3.216277 seconds
minimize_ executed in 3.363983 seconds
minimize_ executed in 3.277673 seconds
compare_on executed in 20.129399 seconds
Generational Distance (GD): (0.017545706797751137, 0.017545706797751137) | (1.9052121879801125, 1.9052121879801125) | (1.905171607431722, 1.905171607431722) | (1.905221614437597, 1.905221614437597) | (2.6943586589142625, 2.6943586589142625) | (2.6943121237290026, 2.6943121237290026)
Generational Distance Plus (GD+): (0.017545706797751137, 0.017545706797751137) | (1.9052121879801125, 1.9052121879801125) | (1.905171607431722, 1.905171607431722) | (1.905221614437597, 1.905221614437597) | (2.6943586589142625, 2.6943586589142625) | (2.6943121237290026, 2.6943121237290026)
Inverted Gene

(array([[-1.90515526]]),
 array([-1.88760955]),
 array([[-1.89042681,  0.        ]]),
 array([[-1.89726225,  0.        ]]),
 array([[-1.88925413,  0.        ]]),
 array([[-1.88683929,  0.        ,  0.        ]]),
 array([[-1.89595136,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=100)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x122044160>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 8.135525 seconds
minimize_ executed in 7.137047 seconds
minimize_ executed in 6.660499 seconds
minimize_ executed in 6.345459 seconds
minimize_ executed in 6.916278 seconds
minimize_ executed in 6.668927 seconds
compare_on executed in 41.865516 seconds
Generational Distance (GD): (0.0007102034615131814, 0.0007102034615131814) | (1.9051580064328715, 1.9051580064328715) | (1.905155547756772, 1.905155547756772) | (1.905156706300228, 1.905156706300228) | (2.6942990191485316, 2.6942990191485316) | (2.6942968589355836, 2.6942968589355836)
Generational Distance Plus (GD+): (0.0007102034615131814, 0.0007102034615131814) | (1.9051580064328715, 1.9051580064328715) | (1.905155547756772, 1.905155547756772) | (1.905156706300228, 1.905156706300228) | (2.6942990191485316, 2.6942990191485316) | (2.6942968589355836, 2.6942968589355836)
Inverted 

(array([[-1.90515526]]),
 array([-1.90444505]),
 array([[-1.90191869,  0.        ]]),
 array([[-1.90410311,  0.        ]]),
 array([[-1.90280549,  0.        ]]),
 array([[-1.90140075,  0.        ,  0.        ]]),
 array([[-1.90358816,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=150)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x1218dc940>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 11.274186 seconds
minimize_ executed in 10.151628 seconds
minimize_ executed in 9.091793 seconds
minimize_ executed in 9.248650 seconds
minimize_ executed in 10.275811 seconds
minimize_ executed in 10.435907 seconds
compare_on executed in 60.479819 seconds
Generational Distance (GD): (0.0006626643243548003, 0.0006626643243548003) | (1.9051578321287934, 1.9051578321287934) | (1.9051555306575372, 1.9051555306575372) | (1.9051564513050057, 1.9051564513050057) | (2.69429789432687, 2.69429789432687) | (2.6942967801028903, 2.6942967801028903)
Generational Distance Plus (GD+): (0.0006626643243548003, 0.0006626643243548003) | (1.9051578321287934, 1.9051578321287934) | (1.9051555306575372, 1.9051555306575372) | (1.9051564513050057, 1.9051564513050057) | (2.69429789432687, 2.69429789432687) | (2.6942967801028903, 2.6942967801028903)
Inver

(array([[-1.90515526]]),
 array([-1.90449259]),
 array([[-1.90202298,  0.        ]]),
 array([[-1.90413454,  0.        ]]),
 array([[-1.90302223,  0.        ]]),
 array([[-1.90232063,  0.        ,  0.        ]]),
 array([[-1.90373013,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=200)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x121e53670>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 16.109201 seconds
minimize_ executed in 14.357476 seconds
minimize_ executed in 12.109358 seconds
minimize_ executed in 12.107415 seconds
minimize_ executed in 14.335851 seconds
minimize_ executed in 13.713381 seconds
compare_on executed in 82.734390 seconds
Generational Distance (GD): (0.000662093067443692, 0.000662093067443692) | (1.9051578308763715, 1.9051578308763715) | (1.9051555289508577, 1.9051555289508577) | (1.9051564459927035, 1.9051564459927035) | (2.6942978810772944, 2.6942978810772944) | (2.6942967777647944, 2.6942967777647944)
Generational Distance Plus (GD+): (0.000662093067443692, 0.000662093067443692) | (1.9051578308763715, 1.9051578308763715) | (1.9051555289508577, 1.9051555289508577) | (1.9051564459927035, 1.9051564459927035) | (2.6942978810772944, 2.6942978810772944) | (2.6942967777647944, 2.6942967777647944)

(array([[-1.90515526]]),
 array([-1.90449316]),
 array([[-1.90202374,  0.        ]]),
 array([[-1.90413773,  0.        ]]),
 array([[-1.90302698,  0.        ]]),
 array([[-1.90233325,  0.        ,  0.        ]]),
 array([[-1.90373455,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=250)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x121bf3430>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 19.241665 seconds
minimize_ executed in 18.572134 seconds
minimize_ executed in 14.880318 seconds
minimize_ executed in 14.618407 seconds
minimize_ executed in 16.152691 seconds
minimize_ executed in 18.238281 seconds
compare_on executed in 101.705154 seconds
Generational Distance (GD): (0.0006620862327171295, 0.0006620862327171295) | (1.9051578308646042, 1.9051578308646042) | (1.9051555289364708, 1.9051555289364708) | (1.9051564459568142, 1.9051564459568142) | (2.6942978801037163, 2.6942978801037163) | (2.6942967777520757, 2.6942967777520757)
Generational Distance Plus (GD+): (0.0006620862327171295, 0.0006620862327171295) | (1.9051578308646042, 1.9051578308646042) | (1.9051555289364708, 1.9051555289364708) | (1.9051564459568142, 1.9051564459568142) | (2.6942978801037163, 2.6942978801037163) | (2.6942967777520757, 2.694296777752

(array([[-1.90515526]]),
 array([-1.90449317]),
 array([[-1.90202375,  0.        ]]),
 array([[-1.90413776,  0.        ]]),
 array([[-1.90302701,  0.        ]]),
 array([[-1.90233418,  0.        ,  0.        ]]),
 array([[-1.90373458,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g16", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G16 object at 0x121977610>>: n_var=5 n_obj=1 n_ieq_constr=38 n_eq_constr=0
minimize_ executed in 23.781144 seconds
minimize_ executed in 22.122681 seconds
minimize_ executed in 18.517024 seconds
minimize_ executed in 19.250420 seconds
minimize_ executed in 24.636367 seconds
minimize_ executed in 20.110341 seconds
compare_on executed in 128.419836 seconds
Generational Distance (GD): (0.0006620861526052124, 0.0006620861526052124) | (1.9051578308645005, 1.9051578308645005) | (1.9051555289362743, 1.9051555289362743) | (1.9051564459566834, 1.9051564459566834) | (2.6942978788326486, 2.6942978788326486) | (2.6942967777519065, 2.6942967777519065)
Generational Distance Plus (GD+): (0.0006620861526052124, 0.0006620861526052124) | (1.9051578308645005, 1.9051578308645005) | (1.9051555289362743, 1.9051555289362743) | (1.9051564459566834, 1.9051564459566834) | (2.6942978788326486, 2.6942978788326486) | (2.6942967777519065, 2.694296777751

(array([[-1.90515526]]),
 array([-1.90449317]),
 array([[-1.90202375,  0.        ]]),
 array([[-1.90413776,  0.        ]]),
 array([[-1.90302701,  0.        ]]),
 array([[-1.9023354,  0.       ,  0.       ]]),
 array([[-1.90373458,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g17", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G17 object at 0x11a762670>>: n_var=6 n_obj=1 n_ieq_constr=0 n_eq_constr=4
minimize_ executed in 15.639013 seconds
minimize_ executed in 14.412090 seconds
minimize_ executed in 15.122250 seconds
minimize_ executed in 14.435750 seconds
minimize_ executed in 13.674852 seconds
minimize_ executed in 13.029664 seconds
compare_on executed in 86.314979 seconds
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[8853.53401644]]),
 array([8935.62490926]),
 array([[8948.39703301,    0.        ]]),
 array([[8864.34241917,    0.        ]]),
 array([[8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ],
        [8865.29995684,    0.        ]]),
 None,
 None)

In [ ]:
pf, *results = test("g18", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G18 object at 0x11ae12e80>>: n_var=9 n_obj=1 n_ieq_constr=13 n_eq_constr=0
minimize_ executed in 17.663278 seconds
minimize_ executed in 15.294040 seconds
minimize_ executed in 11.377968 seconds
minimize_ executed in 11.196920 seconds
minimize_ executed in 14.885971 seconds
minimize_ executed in 15.093739 seconds
compare_on executed in 85.513504 seconds
algo 2 not converge
algo 3 not converge
Performance indicators could not be calculated due to invalid data.


(array([[-0.86573533]]),
 array([-0.58028974]),
 array([[-0.66480897,  0.        ]]),
 None,
 None,
 array([[-0.8636275,  0.       ,  0.       ]]),
 array([[-0.86500238,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g19", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G19 object at 0x11a54fa30>>: n_var=15 n_obj=1 n_ieq_constr=5 n_eq_constr=0
minimize_ executed in 19.506549 seconds
minimize_ executed in 16.172624 seconds
minimize_ executed in 13.971161 seconds
minimize_ executed in 14.166841 seconds
minimize_ executed in 14.937419 seconds
minimize_ executed in 14.966335 seconds
compare_on executed in 93.722320 seconds
Generational Distance (GD): (19.71078594004988, 19.71078594004988) | (33.59722891120456, 33.59722891120456) | (55.22278944851869, 55.22278944851869) | (38.69206766961517, 38.69206766961517) | (54.63314179159451, 54.63314179159451) | (49.133084311528634, 49.133084311528634)
Generational Distance Plus (GD+): (19.71078594004988, 19.71078594004988) | (7.898483371696756, 7.898483371696756) | (44.5327825712308, 44.5327825712308) | (20.753032299188718, 20.753032299188718) | (29.189119208065385, 29.189119208065385) | (16.771537558791756, 16.771537558791756)
Inverted Generational Dis

(array([[32.65559295]]),
 array([52.36637889]),
 array([[40.55407632,  0.        ]]),
 array([[77.18837552,  0.        ]]),
 array([[53.40862525,  0.        ]]),
 array([[61.84471216,  0.        ,  0.        ]]),
 array([[49.42713051,  0.        ,  0.        ]]))

In [ ]:
pf, *results = test("g20", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G20 object at 0x1243fccd0>>: n_var=24 n_obj=1 n_ieq_constr=6 n_eq_constr=14
minimize_ executed in 16.849077 seconds
minimize_ executed in 16.467505 seconds
minimize_ executed in 13.006795 seconds
minimize_ executed in 12.715117 seconds
minimize_ executed in 16.172193 seconds
minimize_ executed in 15.632636 seconds
compare_on executed in 90.845082 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
algo 3 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.0967373]]), None, None, None, None, None, None)

In [ ]:
pf, *results = test("g20", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G20 object at 0x11a6134c0>>: n_var=24 n_obj=1 n_ieq_constr=6 n_eq_constr=14
minimize_ executed in 56.758612 seconds
minimize_ executed in 53.345120 seconds
minimize_ executed in 38.487451 seconds
minimize_ executed in 35.995531 seconds
minimize_ executed in 48.378270 seconds
minimize_ executed in 50.363027 seconds
compare_on executed in 283.329436 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
algo 3 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.0967373]]), None, None, None, None, None, None)

In [ ]:
pf, *results = test("g21", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G21 object at 0x11a7ffc40>>: n_var=7 n_obj=1 n_ieq_constr=1 n_eq_constr=5
minimize_ executed in 15.440804 seconds
minimize_ executed in 13.587099 seconds
minimize_ executed in 13.980542 seconds
minimize_ executed in 15.144594 seconds
minimize_ executed in 14.582725 seconds
minimize_ executed in 14.511071 seconds
compare_on executed in 87.248101 seconds
algo 0 not converge
algo 1 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[193.72451007]]),
 None,
 None,
 array([[3.50720654e-05, 0.00000000e+00]]),
 array([[1.72131059e-05, 0.00000000e+00]]),
 None,
 None)

In [ ]:
pf, *results = test("g21", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G21 object at 0x11a54feb0>>: n_var=7 n_obj=1 n_ieq_constr=1 n_eq_constr=5
minimize_ executed in 53.364665 seconds
minimize_ executed in 43.648621 seconds
minimize_ executed in 48.023121 seconds
minimize_ executed in 49.134275 seconds
minimize_ executed in 45.612665 seconds
minimize_ executed in 43.310189 seconds
compare_on executed in 283.095398 seconds
algo 0 not converge
algo 1 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[193.72451007]]),
 None,
 None,
 array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
      

In [ ]:
pf, *results = test("g22", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G22 object at 0x11a6264c0>>: n_var=22 n_obj=1 n_ieq_constr=1 n_eq_constr=19
minimize_ executed in 17.013247 seconds
minimize_ executed in 15.424247 seconds
minimize_ executed in 16.004501 seconds
minimize_ executed in 17.734283 seconds
minimize_ executed in 15.527695 seconds
minimize_ executed in 15.345007 seconds
compare_on executed in 97.050550 seconds
algo 0 not converge
algo 1 not converge
algo 4 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[236.4309755]]),
 None,
 None,
 array([[22.20156922,  0.        ]]),
 array([[1.36600664, 0.        ],
        [1.36600664, 0.        ]]),
 None,
 None)

In [ ]:
pf, *results = test("g23", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G23 object at 0x11a88af40>>: n_var=9 n_obj=1 n_ieq_constr=2 n_eq_constr=4
minimize_ executed in 15.933207 seconds
minimize_ executed in 14.570665 seconds
minimize_ executed in 15.721558 seconds
minimize_ executed in 16.205201 seconds
minimize_ executed in 14.973595 seconds
minimize_ executed in 15.523515 seconds
compare_on executed in 92.928815 seconds
algo 0 not converge
algo 1 not converge
algo 5 not converge
Performance indicators could not be calculated due to invalid data.


(array([[-400.]]),
 None,
 None,
 array([[-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
        [-3900.,     0.],
     

In [ ]:
pf, *results = test("g24", n_gen=300)
pf, results[0].F, results[1].F, results[2].F, results[3].F, results[4].F, results[5].F

Problem <bound method Problem.name of <pymoo.problems.single.g.G24 object at 0x11aa66dc0>>: n_var=2 n_obj=1 n_ieq_constr=2 n_eq_constr=0
minimize_ executed in 18.484372 seconds
minimize_ executed in 16.285200 seconds
minimize_ executed in 15.010937 seconds
minimize_ executed in 14.520960 seconds
minimize_ executed in 15.814033 seconds
minimize_ executed in 16.029151 seconds
compare_on executed in 96.145890 seconds
Generational Distance (GD): (0.000429234313505944, 0.000429234313505944) | (5.508013945277377, 5.508013945277377) | (5.508013406918499, 5.508013406918499) | (5.508013900618469, 5.508013900618469) | (7.789507070576186, 7.789507070576186) | (7.789507076953684, 7.789507076953684)
Generational Distance Plus (GD+): (0.000429234313505944, 0.000429234313505944) | (5.508013945277377, 5.508013945277377) | (5.508013406918499, 5.508013406918499) | (5.508013900618469, 5.508013900618469) | (7.789507070576186, 7.789507070576186) | (7.789507076953684, 7.789507076953684)
Inverted Generationa

(array([[-5.50801327]]),
 array([[-5.50758404],
        [-5.50758404],
        [-5.50758404],
        [-5.50758404]]),
 array([[-5.50528907,  0.        ]]),
 array([[-5.50679232,  0.        ]]),
 array([[-5.50538091,  0.        ]]),
 array([[-5.50796411,  0.        ,  0.        ]]),
 array([[-5.50769425,  0.        ,  0.        ]]))

# SRN


In [ ]:
pf, *results = test("srn", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(results[0].F[:, :-1], marker="d", color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[:, :-1],
    marker="1",
    color="blue",
    label="UMOP + Our generalized framework",
)
plot.add(
    results[2].F[:, :-1], marker="2", color="purple", label="UMOP + Our framework + CDF"
)
plot.show()

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0][:, :-1], marker="d", color="yellow")
plot.add(results[2].F[:, :-1], marker="+", color="green")
plot.add(results[3].F[:, :-1], marker="1", color="blue")
plot.add(results[4].F[:, :-1], marker="2", color="purple")
plot.show()

# ZDT


In [ ]:
# problem_zdt1 = get_problem("zdt1")
# problem = problem_zdt1
# print(
#     f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
# )

# res0, res1, res2, res3 = compare_on(problem)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt2 = get_problem("zdt2")

# res0, res1, res2, res3 = compare_on(problem_zdt2)

# plot = Scatter()
# plot.add(problem_zdt2.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt3 = get_problem("zdt3")

# res0, res1, res2, res3 = compare_on(problem_zdt3)

# plot = Scatter()
# plot.add(problem_zdt3.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.Fa, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt4 = get_problem("zdt4")

# res0, res1, res2, res3 = compare_on(problem_zdt4)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem_zdt4.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt5 = get_problem("zdt5", normalize=False)

# res0, res1, res2, res3 = compare_on(problem_zdt5, n_gen=3000)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem_zdt5.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt6 = get_problem("zdt6")

# res0, res1, res2, res3 = compare_on(problem_zdt6)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem_zdt6.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# cv = res3.F[:, 0]
# least_infeas = cv.argmin()
# x = res3.X[least_infeas]

# sol = Individual(X=x)
# Evaluator().eval(problem2, sol)

# print("Best solution found: \nX = %s\nF = %s\nCV = %s" % (sol.X, sol.F, sol.CV))

In [ ]:
# plot = Scatter()
# plot.add(problem2.pareto_front(), marker="*", color="black", alpha=0.7, s=100)
# plot.add(res3.F[:,1:], facecolor="none", edgecolor="red")
# plot.show()

In [ ]:
# X, F = res3.opt.get("X", "F")
# hist = res3.history
# print(len(hist))

In [ ]:
# n_evals = []             # corresponding number of function evaluations\
# hist_F = []              # the objective space values in each generation
# hist_cv = []             # constraint violation in each generation
# hist_cv_avg = []         # average constraint violation in the whole population

# for algo in hist:

#     # store the number of function evaluations
#     n_evals.append(algo.evaluator.n_eval)

#     # retrieve the optimum from the algorithm
#     opt = algo.opt

#     # store the least contraint violation and the average in each population
#     hist_cv.append(opt.get("CV").min())
#     hist_cv_avg.append(algo.pop.get("CV").mean())

#     # filter out only the feasible and append and objective space values
#     feas = np.where(opt.get("feasible"))[0]
#     hist_F.append(opt.get("F")[feas])

In [ ]:
# k = np.where(np.array(hist_cv) <= 0.0)[0].min()
# print(f"At least one feasible solution in Generation {k} after {n_evals[k]} evaluations.")

In [ ]:
# # replace this line by `hist_cv` if you like to analyze the least feasible optimal solution and not the population
# vals = hist_cv_avg

# k = np.where(np.array(vals) <= 0.0)[0].min()
# print(f"Whole population feasible in Generation {k} after {n_evals[k]} evaluations.")

# plt.figure(figsize=(7, 5))
# plt.plot(n_evals, vals,  color='black', lw=0.7, label="Avg. CV of Pop")
# plt.scatter(n_evals, vals,  facecolor="none", edgecolor='black', marker="p")
# plt.axvline(n_evals[k], color="red", label="All Feasible", linestyle="--")
# plt.title("Convergence")
# plt.xlabel("Function Evaluations")
# plt.ylabel("Hypervolume")
# plt.legend()
# plt.show()

In [ ]:
# approx_ideal = F[:,-2:].min(axis=0)
# approx_nadir = F[:,-2:].max(axis=0)
# approx_ideal, approx_nadir

In [ ]:
# from pymoo.indicators.hv import Hypervolume

# metric = Hypervolume(ref_point=np.array([0.1, 0.1]),
#                      norm_ref_point=False,
#                      zero_to_one=True,
#                      ideal=approx_ideal,
#                      nadir=approx_nadir)

# hv = [metric.do(_F[:, -2:]) for _F in hist_F]

# plt.figure(figsize=(7, 5))
# plt.plot(n_evals, hv, color='black', lw=0.7, label="Avg. CV of Pop")
# plt.scatter(n_evals, hv, facecolor="none", edgecolor='black', marker="p")
# plt.title("Convergence")
# plt.xlabel("Function Evaluations")
# plt.ylabel("Hypervolume")
# plt.show()

In [ ]:
# from pymoo.util.running_metric import RunningMetricAnimation

# running = RunningMetricAnimation(delta_gen=5,
#                         n_plots=3,
#                         key_press=False,
#                         do_show=True)

# for algorithm in res3.history[:50]:
#     running.update(algorithm)

In [ ]:
# from pymoo.indicators.igd import IGD

# metric = IGD(pf, zero_to_one=True)

# igd = [metric.do(_F) for _F in hist_F]

# plt.plot(n_evals, igd,  color='black', lw=0.7, label="Avg. CV of Pop")
# plt.scatter(n_evals, igd,  facecolor="none", edgecolor='black', marker="p")
# plt.axhline(10**-2, color="red", label="10^-2", linestyle="--")
# plt.title("Convergence")
# plt.xlabel("Function Evaluations")
# plt.ylabel("IGD")
# plt.yscale("log")
# plt.legend()
# plt.show()

# OSY


In [ ]:
pf, *results = test("osy", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0][:, :-1], marker="d", color="yellow")
plot.add(results[2].F[:, :-1], marker="+", color="green")
plot.add(results[3].F[:, :-1], marker="1", color="blue")
plot.add(results[4].F[:, :-1], marker="2", color="purple")
plot.show()

# TNK


In [ ]:
pf, *results = test("tnk", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(results[0].F[:, :-1], marker="d", color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[:, :-1],
    marker="1",
    color="blue",
    label="UMOP + Our generalized framework",
)
plot.add(
    results[2].F[:, :-1], marker="2", color="purple", label="UMOP + Our framework + CDF"
)
plot.show()

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0][:, :-1], marker="d", color="yellow")
plot.add(results[2].F[:, :-1], marker="+", color="green")
plot.add(results[3].F[:, :-1], marker="1", color="blue")
plot.add(results[4].F[:, :-1], marker="2", color="purple")
plot.show()

In [ ]:
# plt.hist(res1.F[:, -1:], bins=30, color="green", alpha=0.7, label="Sum CV_as_obj")
# plt.hist(res2.F[:, -1:], bins=30, color="blue", alpha=0.7, label="Max CV_as_obj")
# plt.hist(res3.F[:, -1:], bins=30, color="purple", alpha=0.7, label="Median CV_as_obj")

# Add labels and title
# plt.xlabel("Values")
# plt.ylabel("Frequency")
# plt.title("Histogram of CV")

# Add a legend
# plt.legend()

# Show the plot
# plt.show()

# Carside


In [ ]:
pf, *results = test("Carside", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(
    results[0].F[results[0].F[:, -1] == 0][:, :-1],
    marker="d",
    color="yellow",
    label="UMOP + Baseline",
)
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[results[1].F[:, -1] == 0][:, :-1],
    marker="1",
    color="blue",
    label="UMOP + Our generalized framework",
)
plot.add(
    results[2].F[results[2].F[:, -1] == 0][:, :-1],
    marker="2",
    color="purple",
    label="UMOP + Our framework + CDF",
)
plot.show()

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(
    results[1].F[results[1].F[:, -1] == 0][:, :-1],
    marker="d",
    color="yellow",
    label="UMOP + Baseline",
)
plot.add(
    results[2].F[:, :-1], marker="+", color="green", label="UMOP + Baseline + Our CDP"
)
plot.add(
    results[3].F[:, :-1],
    marker="1",
    color="blue",
    label="UMOP + Our generalized framework",
)
plot.add(
    results[4].F[:, :-1], marker="2", color="purple", label="UMOP + Our framework + CDF"
)
plot.show()

In [ ]:
plot = Scatter(tight_layout=True)
# plot.add(results[1].F, marker="+", color="green", label="Baseline")
plot.add(results[2].F, marker=".", color="green", label="Baseline + Our CDP")
plot.add(results[3].F, marker="1", color="blue",
         label="Our generalized framework")
plot.add(results[4].F, marker="1", color="purple", label="Our framework + CDF")
plot.show()

# Pressure Vessel


In [ ]:
pf, *results = test("pressure_vessel", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

# G


## G1


In [ ]:
pf, *results = test("g1", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
plot.add(
    results[1].F[results[1].F[:, -1] == 0][:, :-1],
    marker="D",
    color="Yellow",
    label="Baseline",
)
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

Exception: Inputs with different dimensions were added: [1 2]

## G2


In [ ]:
pf, *results = test("g1", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0][:, :-1], marker="D", color="yellow")
plot.add(results[2].F[:, :-1], marker="+", color="green")
plot.add(results[3].F[:, :-1], marker="1", color="blue")
plot.add(results[4].F[:, :-1], marker="1", color="purple")
plot.show()

## G3 (1 Constraint)


In [ ]:
# pf, res0, res1, res2, res3 = test("g3", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G4


In [ ]:
pf, res0, res1, res2, res3 = test("g4", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G5


In [ ]:
pf, res0, res1, res2, res3 = test("g5", n_gen=300)

In [ ]:
pf

In [ ]:
res0.F

In [ ]:
res2.F

In [ ]:
res3.F

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G6


In [ ]:
pf, res0, res1, res2, res3 = test("g6", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G7


In [ ]:
pf, res0, res1, res2, res3 = test("g7", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G8


In [ ]:
pf, res0, res1, res2, res3 = test("g8", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G9


In [ ]:
pf, res0, res1, res2, res3 = test("g9", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G10


In [ ]:
pf, res0, res1, res2, res3 = test("g10", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G11


In [ ]:
pf, res0, res1, res2, res3 = test("g11", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G12


In [ ]:
pf, res0, res1, res2, res3 = test("g12", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G13


In [ ]:
pf, res0, res1, res2, res3 = test("g13", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## G14


In [ ]:
pf, res0, res1, res2, res3 = test("g14", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

# MW


## MW1 (1 constraint)


In [ ]:
pf, *results = test("mw", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW2


In [ ]:
pf, *results = test("mw2", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW3


In [ ]:
pf, *results = test("mw3", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW4 (1 constraint)


In [ ]:
pf, *results = test("mw4", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW5


In [ ]:
pf, *results = test("mw5", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW6 (1 constraint)


In [ ]:
pf, *results = test("mw6", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW7


In [ ]:
pf, *results = test("mw7", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW8 (1 constraint)


In [ ]:
pf, *results = test("mw8", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW9


In [ ]:
pf, *results = test("mw9", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW10


In [ ]:
pf, *results = test("mw13", n_gen=15)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## MW14 (1 constraint)


In [ ]:
pf, *results = test("mw14", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
# plot.add(results[3].F[:, :-1], marker="1", color="blue",
#          label="Our generalized framework")
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

# DAS-CMOP


## DAS-CMOP-1


In [ ]:
pf, *results = test("dascmop1", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop2", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop3", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop4", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop5", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop6", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop7", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop8", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop9", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

# MODAct


In [ ]:
pf, *results = test("modact", 10, "cs1")

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
terminate_here

# To_excel


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from constraints.as_obj import CVRAsObjective, CDFAsObjective

POP_SIZE = 500
N_GEN = 300

core_moea1 = NSGA2(pop_size=POP_SIZE, survival=RankAndCrowding())
core_moea2 = NSGA2(pop_size=POP_SIZE, survival=MyConstrRankAndCrowding())
core_moea3 = NSGA2(pop_size=POP_SIZE, survival=ParallelConstrRankAndCrowding())


@performance_timer
def compare_on(problem, n_gen=300):
    # Configuration for each optimization strategy
    configurations = [
        # (problem, core_moea1, ("n_gen", n_gen), {"seed": 1, "verbose": False}),
        (ConstraintsAsObjective(problem), core_moea1, ("n_gen", n_gen), {"seed": 1}),
        (CVRAsObjective(problem), core_moea1, ("n_gen", n_gen), {"seed": 1}),
        (CDFAsObjective(problem), core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (ConstraintsAsObjective(problem),
        #  core_moea2, ("n_gen", n_gen), {"seed": 1}),
        # (
        #     CVRAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (
        #     CDFAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
    ]

    results = []
    for config in configurations:
        result = minimize_(*config[:-1], **config[-1])
        results.append(result)
    return results


def test(problem_name, n_gen=300, **kwargs):
    if "dascmop" in problem_name or "modact" in problem_name:
        if "additional_args" in kwargs:
            problem = get_problem(problem_name, kwargs["additional_args"])
        else:
            raise ValueError(
                f"Additional arguments required for problem {problem_name}"
            )
    else:
        problem = get_problem(problem_name)
    pf = problem.pareto_front()
    print(
        f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
    )

    results = compare_on(problem, n_gen=n_gen)

    # Check if any of the results is None
    if any(result is None for result in results):
        print("One or more optimization runs failed to return a result.")
        return pf, [None] * len(results)

    # Prepare data for performance indicators
    prepared_data = []
    for i, result in enumerate(results):
        if result and hasattr(result, "F") and result.F is not None:
            # if i > 0:
            if True:
                prepared_data.append(result.F[:, :-1])
            else:
                prepared_data.append(result.F)
        else:
            prepared_data.append(None)

    # Ensure all data is valid before calculating performance indicators
    if all(data is not None for data in prepared_data):
        performance_results = performance_indicators(pf, prepared_data)
        for i, metric in enumerate(metric_classes.keys()):
            print(
                f"{metric}: " + " | ".join(f"{perf[i]}" for perf in performance_results)
            )
    else:
        print("Performance indicators could not be calculated due to invalid data.")

    return pf, results


def create_charts(df, output_dir="."):
    metrics = df["Metric"].unique()
    problems = df["Problem"].unique()

    for problem in problems:
        problem_df = df[df["Problem"] == problem]
        for metric in metrics:
            metric_df = problem_df[problem_df["Metric"] == metric]
            plt.figure(figsize=(10, 6))
            plt.bar(
                metric_df["Algorithm"],
                metric_df["GD+ Mean"],
                yerr=metric_df["GD+ Median"],
            )
            plt.title(f"{metric} for {problem}")
            plt.xlabel("Algorithm")
            plt.ylabel(metric)
            plt.savefig(f"{output_dir}/{problem}_{metric}.png")
            plt.close()


# Function to compare multiple problems and export results to Excel


def compare_problems_to_excel(problem_configs, n_gen=300, output_file="results.xlsx"):
    results_list = []
    for problem_config in problem_configs:
        additional_args = None
        if isinstance(problem_config, tuple):
            problem_name, additional_args = problem_config
            pf, results = test(
                problem_name, n_gen=n_gen, additional_args=additional_args
            )
        else:
            problem_name = problem_config
            pf, results = test(problem_name, n_gen=n_gen)

        # Skip problems where results are None
        if results[0] is None:
            continue

        # Compute performance indicators
        prepared_data = []
        for i, result in enumerate(results):
            if result and hasattr(result, "F") and result.F is not None:
                # if i > 0:
                if True:
                    prepared_data.append(result.F[:, :-1])
                else:
                    prepared_data.append(result.F)
            else:
                prepared_data.append(None)

        if all(data is not None for data in prepared_data):
            performance_results = performance_indicators(pf, prepared_data)
            for i, rs in enumerate(performance_results):
                (
                    (gd_mean, gd_median),
                    (gd_plus_mean, gd_plus_median),
                    (igd_mean, igd_median),
                    (igd_plus_mean, igd_plus_median),
                ) = rs
                result_row = {
                    "Problem": (
                        problem_name
                        if additional_args is None
                        else f"{problem_name} {additional_args}"
                    ),
                    "Algorithm": f"Algorithm {i+1}",
                    "GD Mean": gd_mean,
                    "GD Median": gd_median,
                    "GD+ Mean": gd_plus_mean,
                    "GD+ Median": gd_plus_median,
                    "IGD Mean": igd_mean,
                    "IGD Median": igd_median,
                    "IGD+ Mean": igd_plus_mean,
                    "IGD+ Median": igd_plus_median,
                    "Time": (
                        results[i].exec_time
                        if hasattr(results[i], "exec_time")
                        else np.nan
                    ),
                }
                results_list.append(result_row)

    # Create DataFrame and export to Excel
    df = pd.DataFrame(results_list)
    df.to_excel(output_file, index=False)
    print(f"Results exported to {output_file}")

    # Create charts for visualizing the results
    # create_charts(df)


# Example usage
problem_configs = [
    # "wrm",
    # "srn",
    # "clutch"
]

problem_dascmop = [f"dascmop{i}" for i in range(1, 10)]
for i in range(16, 17):
    for p in problem_dascmop:
        problem_configs.append((p, i))
compare_problems_to_excel(
    problem_configs,
    n_gen=100,
    output_file=f"optimization_results_{datetime.now().timestamp()}.xlsx",
)

In [ ]:
problem_configs = [
    "g7",
    "g8",
    "g9",
    # "mw7",
    # "mw7",
    # "mw9",
    # "mw10",
]
# problem_configs += [f'g{i}' for i in range(1,25)]
compare_problems_to_excel(
    problem_configs,
    n_gen=500,
    output_file=f"optimization_results_{datetime.now().timestamp()}.xlsx",
)

# Diversity Factor


In [ ]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.problems import get_problem
from scipy.spatial import cKDTree
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover as SBX
from pymoo.operators.mutation.pm import PolynomialMutation as PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize
from pymoo.core.callback import Callback

In [ ]:
from annoy import AnnoyIndex


class DiversityEnhancedProblem(Problem):
    def __init__(self, problem, neighbors=5, trees=10):
        super().__init__(
            n_var=problem.n_var,
            n_obj=problem.n_obj + 1,
            n_constr=problem.n_constr,
            xl=problem.xl,
            xu=problem.xu,
        )
        self.base_problem = problem
        self.neighbors = neighbors
        self.population = []
        self.tree = AnnoyIndex(self.n_var, "manhattan")
        self.tree_cache = {}  # To store diversity values
        self.trees = trees  # Number of trees for Annoy

    def update_population(self, pop):
        self.population = pop.get("X")
        self.tree.unbuild()  # Clear the current tree
        for i, x in enumerate(self.population):
            self.tree.add_item(i, x.tolist())
        self.tree.build(self.trees)
        self.tree_cache = {}  # Clear cache after update

    def calculate_diversity(self, x):
        if len(self.population) > 1:
            distances = self.tree.get_nns_by_vector(
                x, self.neighbors, include_distances=True
            )[1]
            return 1.0 / np.mean(distances)
        return 0

    def _evaluate(self, x, out, *args, **kwargs):
        base_out = {}
        self.base_problem._evaluate(x, base_out, *args, **kwargs)
        f = base_out["F"]
        x = np.atleast_2d(x)

        # Calculate diversity, using cache if available
        diversity = np.array(
            [self.tree_cache.get(
                tuple(xi), self.calculate_diversity(xi)) for xi in x]
        )
        for xi, div in zip(x, diversity):
            self.tree_cache[tuple(xi)] = div  # Update cache

        out["F"] = np.column_stack([f, diversity])


class DiversityCallback(Callback):
    def __init__(self, problem):
        super().__init__()
        self.problem = problem

    def notify(self, algorithm):
        self.problem.update_population(algorithm.pop)


# Original problem from pymoo
original_problem = get_problem("zdt2")

# Enhanced problem with diversity objective
problem = DiversityEnhancedProblem(original_problem, neighbors=10)

# Setup the algorithm
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)

# Execute the optimization
res1 = minimize(
    problem,
    algorithm,
    ("n_gen", 10000),
    verbose=True,
    seed=1,
    callback=DiversityCallback(problem),
)

res2 = minimize(
    original_problem,
    algorithm,
    ("n_gen", 2000),
    verbose=False,
    seed=0,
    # callback=DiversityCallback(problem)
)

In [ ]:
print(res1.F.shape, res2.F.shape)

In [ ]:
from visualization.scatter import Scatter

plot = Scatter()
plot.add(res1.F[:, :-1], facecolor="none", edgecolor="red")
plot.add(res2.F, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
from visualization.scatter import Scatter

plot = Scatter()
plot.add(res1.F, facecolor="none", edgecolor="red")
# plot.add(res2.F, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
from pymoo.factory import get_performance_indicator

# Assuming 'result' is the output of your optimization algorithm
hv = get_performance_indicator(
    "hv", ref_point=np.array([1.1, 1.1])
)  # Adjust ref_point as needed
hypervolume = hv.calc(res1.F)

print("Hypervolume:", hypervolume)

In [ ]:
from visualization.scatter import Scatter
from annoy import AnnoyIndex
import numpy as np


class DiversityEnhancedProblem2(Problem):
    def __init__(
        self, problem, neighbors=5, trees=10, update_interval=5, change_threshold=0.2
    ):
        super().__init__(
            n_var=problem.n_var,
            n_obj=problem.n_obj + 1,
            n_constr=problem.n_constr,
            xl=problem.xl,
            xu=problem.xu,
        )
        self.base_problem = problem
        self.neighbors = neighbors
        self.population = []
        self.tree = AnnoyIndex(self.n_var, "euclidean")
        self.trees = trees
        self.update_interval = update_interval
        self.change_threshold = change_threshold
        self.generation_count = 0
        self.last_population = None

    def update_population(self, pop):
        self.population = pop.get("X")
        self.generation_count += 1

        # Determine if the tree needs updating
        if (
            self.last_population is None
            or self.generation_count % self.update_interval == 0
            or self.population_has_changed()
        ):
            self.rebuild_tree()
            self.last_population = np.array(self.population.copy())
            print("Tree rebuilt.")

    def population_has_changed(self):
        """Checks if significant changes have occurred in the population"""
        if self.last_population is not None:
            dists = np.linalg.norm(self.last_population - self.population, axis=1)
            significant_changes = np.mean(dists > self.change_threshold)
            # More than 10% of the population has significantly changed
            return significant_changes > 0.1
        return False

    def rebuild_tree(self):
        """Rebuilds the Annoy index"""
        self.tree.unbuild()
        for i, x in enumerate(self.population):
            self.tree.add_item(i, x.tolist())
        self.tree.build(self.trees)

    def calculate_diversity(self, x):
        if len(self.population) > 1:
            distances = self.tree.get_nns_by_vector(
                x, self.neighbors, include_distances=True
            )[1]
            return 1.0 / np.mean(distances)
        return 0

    def _evaluate(self, x, out, *args, **kwargs):
        base_out = {}
        self.base_problem._evaluate(x, base_out, *args, **kwargs)
        f = base_out["F"]
        x = np.atleast_2d(x)

        # Calculate diversity
        diversity = np.array([self.calculate_diversity(xi) for xi in x])
        out["F"] = np.column_stack([f, diversity])


# Original problem from pymoo
original_problem = get_problem("zdt2")

# Enhanced problem with diversity objective
problem2 = DiversityEnhancedProblem2(original_problem, neighbors=10)

# Setup the algorithm
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)

# Execute the optimization
res1 = minimize(
    problem2,
    algorithm,
    ("n_gen", 5000),
    verbose=True,
    seed=1,
    callback=DiversityCallback(problem),
)

res2 = minimize(
    original_problem,
    algorithm,
    ("n_gen", 2000),
    verbose=False,
    seed=0,
)


plot = Scatter()
plot.add(res1.F[:, 1:], facecolor="none", edgecolor="red")
plot.add(res2.F, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
from pymoo.core.problem import Problem
import numpy as np


class ConstraintAsObjectiveProblem(Problem):
    def __init__(self, problem):
        super().__init__(
            n_var=problem.n_var,
            n_obj=problem.n_obj + problem.n_constr,
            n_constr=0,  # No explicit constraints as they are now objectives
            xl=problem.xl,
            xu=problem.xu,
        )
        self.problem = problem

    def _evaluate(self, x, out, *args, **kwargs):
        # Evaluate the original problem
        res = {}
        self.problem._evaluate(x, res)

        # Combine original objectives with constraints as new objectives
        f = res["F"]
        # Constraint violation for g(x) > 0
        g = np.where(res["G"] > 0, res["G"], 0)
        h = np.abs(res["H"])  # Absolute value for equality constraints

        # Combine into a single objective array
        out["F"] = np.column_stack([f, g, h])

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.selection.tournament import compare
from pymoo.optimize import minimize
from pymoo.problems import get_problem


class FeasibilityFirstNSGA2(NSGA2):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _next(self):
        # Overriding the selection process to prioritize feasibility
        def feasibility_first(a, b):
            # Check if either solution is feasible
            is_feasible_a = np.all(self.pop[a].get(
                "F")[self.problem.n_obj:] <= 0)
            is_feasible_b = np.all(self.pop[b].get(
                "F")[self.problem.n_obj:] <= 0)

            if is_feasible_a and not is_feasible_b:
                return a
            elif not is_feasible_a and is_feasible_b:
                return b
            else:
                # If both are feasible or infeasible, fall back to regular comparison
                return compare(
                    a, b, self.pop, self.n_survive, self.rank_key, self.crowding_key
                )

        # Tournament selection modified for feasibility
        mating_pool = self.selection(
            self.pop, self.n_offsprings, feasibility_first)
        off = self.crossover.do(self.problem, self.pop, mating_pool)
        off = self.mutation.do(self.problem, off)
        self.evaluator.eval(self.problem, off)
        self.pop = self.survival.do(
            self.problem, self.pop, off, n_survive=self.pop_size
        )


problem = get_problem("zdt1")
wrapped_problem = ConstraintAsObjectiveProblem(problem)

algorithm = FeasibilityFirstNSGA2(pop_size=100)

res = minimize(wrapped_problem, algorithm, ("n_gen", 100), verbose=True)